In [ ]:
# Running the Packages and parameters script which imports all the necessary packages and speifies parmeters for plotting
# Also specifying the order of the treatments when plotting so they all get the same color in every graph
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import numpy as np
import os
from scipy import spatial
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from skimage import feature
from skimage import morphology
import pickle
from scipy import stats
from scipy import signal
import time
import importlib
import h5py
import fl
import warnings
import sys
#from termcolor import colored
from multiprocessing import Pool
from nd2reader import ND2Reader
#from scipy.misc import imread
import statsmodels.api as sm
from pathlib import Path
from scipy.stats import sem

#Parameters for export to Illustrator
#---------------------------------------------------------------------------
#matplotlib.rcParams['pdf.fonttype'] = 42
#matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
#plt.rcParams['interpolation'] = 'none'
#---------------------------------------------------------------------------




#Plot parameters & Styles
#---------------------------------------------------------------------------
#sns.set_style('darkgrid')
#sns.set_style('whitegrid')
sns.set_style('ticks')
#sns.set_context("talk") #paper,notebook,talk,poster
sns.set_context("poster")
plt.rcParams['image.cmap'] = 'jet'
pal_bw='Greys'
pal_few_c='Set1'
pal_many_c='muted'
plot_height=6
plot_aspect=1.25
ci_1=70
time_lim=[0,600]
time_ticks=[0,250,500]
title_pos=0.9
title_size=25
#---------------------------------------------------------------------------
# Orders of variables for plotting 


#Create lists that specify the order of the variable values in the graphs
mov_class_order=['still','contractile','loose','none']
mov_class_order1=['still','contractile','loose']
movement_order=['immobile','contracting','drifting','unstable','none']
movement_order1=['immobile','contracting','drifting','unstable']
movements_order2=['immobile','contracting','drifting']
position_order=['head','tail','outside']

#inh_order1=['ctrl','biva','par4--','cang','par4--biva']
inh_order=['ctrl','saline','biva','par4--','cang','par4--biva']
inh_order1=['saline','biva','par4--','cang','par4--biva']
inh_order_thrombin=['saline','biva','par4--','par4--biva']
inh_labels=['Control','Saline','Bivalirudin','PAR4-/-','Cangrelor','PAR4-/- + Bivalirudin']
inh_labels1=['Saline','Bivalirudin','PAR4 -/-','Cangrelor','PAR4 -/- + Bivalirudin']
bol_order=[True,False]
#pal = dict(zip(days, sns.color_palette("Set1", n_colors=len(days))))



importlib.reload(fl)
%matplotlib inline
warnings.simplefilter(action = "ignore", category = FutureWarning)
processors=1

In [ ]:
plt.rcParams['image.cmap'] = 'jet'
os.getcwd()

In [ ]:
path = Path('/Users/jni/projects/pia-tracking/')# PIA Manually change the path to the folder including the outputdata
#For some reason the path name requires double backslashes here.

dirs = [e for e in path.iterdir() if e.is_dir()]
for c,e in enumerate (dirs):
    print(c,e)

In [ ]:
# PIA specify which of the dirs listed above that the script should use (the one containing the pickle files from script 1)

data_path = dirs[2]

# C:\Users\piamaril\Python\Scripts\200823_Test\20200823-112806
# C:\Users\piamaril\Python\Scripts\200612\20200613-214101
#'D:/Forskning/Tracking/Intravital/Demo Injury for Python/200116'
#'C:/Users/Human/Google Drive/IVM sharing/IVM Tracking/Data/Demo experiments 200114'
#'D:/Forskning/Tracking/Intravital/Demo Injury for Python'
#C:/Users/Lars2/Desktop/IVM sharing/IVM Tracking/Data'

In [ ]:
files=fl.get_objectDF_files(dirs[2]) # PIA change to correct dirs[x] here too
#'C:\\Users\\Human\\Google Drive\\IVM sharing\\IVM Tracking\\Scripts\\New fl 200116')#dirs[3])
print(f"There are {len(files)} pointcloud files in the data_path \nList of files in data_path:")
for c, value in enumerate(files,0):
    print(c, value)

In [ ]:
# PIA change the range depending on how many injuries in data set. If a test injury use flist=[0] 
#If a list of injuries of example 66 injuries: write (1,65,1) - IS THIS CORRECT??? (0,65,1)???
#Put hashtag on the one that you are not using
flist=[0]
#flist=list(range(1,65,1))

In [ ]:
ls_df=[]
for n in flist:
    pc_=pd.read_pickle(files[n])
    print(f"Observations in file number {n} = {pc_.shape[0]}")
    ls_df.append(pc_)
pc=pd.concat(ls_df)
print(f"Total number of observations = {pc.shape[0]}")
print(f"Average number of observations per exp = {int(pc.shape[0]/len(flist))}")

In [ ]:
pc.head(5).T

In [ ]:
print('Columns ', pc.columns, '\nDimensions ',pc.shape, '\nFrames ', len(pd.unique(pc.frame)))

# Load PC collection

In [ ]:
print(pd.unique(pc.path))

# Ytterligare Filter 

In [ ]:
#help(np.geomspace)

In [ ]:
# OM DET BEHÖVS, renso objekt från mindre objekt (skräp), och se hur många objekt före och efter


print(f"{pc.shape[0]} \t\t objects\n")
for n in list(np.around(np.geomspace(4, 1024, num=9)).astype(int)):
        if n<129:
            print(f"{len(pc[pc['size']<n])} \t\t smaller than \t\t {n} \t\t pixels")
        else:
            print(f"{len(pc[pc['size']>n])} \t\t larger than \t\t {n} \t\t pixels")
#print(f"{len(pc[pc['size']>1024])} \t\t larger than \t\t 1024 \t\t pixels")
    #print(f"There are {k}")
    #print('hello')  

    
    
    

# Reset index, drop columns

In [ ]:
pc['pid']=list(range(pc.shape[0]))

In [ ]:
pc.head(3).T

In [ ]:
pc.tail(3).T


In [ ]:
pc.set_index('pid')

In [ ]:
pc[['size','c0_max','c1_max']].describe()

In [ ]:
pc.tail(3).T

In [ ]:
pc.head()

In [ ]:
pc.shape


# Tracking

In [ ]:
import trackpy as tp

In [ ]:
# Function for tracking. Seach range = 3 um with these settings - change?
def track(pct):
    search_range = 3 #4
    #pred = tp.predict.NearestVelocityPredict()#Predikterar position utifran tidigare hastighet
    #linked_pc = pred.link_df(pct, search_range, pos_columns=['xs', 'ys', 'zs'], memory=1)
    linked_pc = tp.link_df(pct, search_range, pos_columns=['xs', 'ys', 'zs'], memory=1)
    #pred = trackpy.predict.NearestVelocityPredict()
    #tr = pred.link_df(pandas.concat(frames), 0.5)
    return(linked_pc)

In [ ]:
# Applying tracking function to find trajectories ?
t_grp=pc.reset_index().groupby(['path']).apply(track)


In [ ]:
t_grp.head()

In [ ]:
tidy = t_grp.drop(['path'], axis=1).reset_index().drop(['level_1'], axis=1)

In [ ]:
tidy.to_csv('/Users/jni/Dropbox/share-files/tracks.csv', header=True)

In [ ]:
tracks=t_grp.drop(['path'], axis=1).reset_index().sort_values('pid')

In [ ]:
dv_=[]

p_grp=tracks.groupby(['path', 'particle'])
for i, gr in p_grp:
    grp=gr.sort_values('frame')
    dvx = np.append(np.diff(grp['xs']), np.nan)
    dvy = np.append(np.diff(grp['ys']), np.nan)
    dvz = np.append(np.diff(grp['zs']), np.nan)
    df=grp.pid.to_frame()
    df['dvx']=dvx
    df['dvy']=dvy
    df['dvz']=dvz
    df['particle']=grp.particle
    
    dv_.append(df)

In [ ]:
dv=pd.concat(dv_, axis=0)

In [ ]:
dv=dv.sort_values('pid')

In [ ]:
dv['dv']=(dv.dvx**2+dv.dvy**2+dv.dvz**2)**0.5

In [ ]:
tracks_grp=dv.sort_values('pid').reset_index()
tracks_grp=tracks_grp[['pid', 'dvx', 'dvy', 'dvz', 'dv', 'particle']].set_index('pid')

In [ ]:
tracks_grp.tail(20)

# Nearest Neighbours

In [ ]:
def nb(pc):
    nb_count=3
    key_dist={}
    key_idx={}
    #print(len(pc))
    p1i=pc.reset_index().pid
    if len(pc)>nb_count:
        
        dmap=spatial.distance.squareform(spatial.distance.pdist(pc[['xs','ys','zs']].values))
        dmap_sorted=np.sort(dmap, axis=0)
        dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in range(nb_count):

            nb_dist=(dmap_sorted[i+1,:])
            nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nb_d_' + str(i)]=nb_dist
            key_idx['nb_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        key_idx.update(key_dist)
    else:
        a = np.empty((len(pc)))
        a[:] = np.nan
        key_idx['nb_i_0']=a
    
    df=pd.DataFrame(key_idx)
    df=pd.concat([p1i, df], axis=1)
    
    return df


In [ ]:
t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nb).reset_index()

In [ ]:
t_grp=t_grp.set_index('pid')

In [ ]:
t_grp.head(3)#.T

In [ ]:
nb_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)

In [ ]:
nb_grp.head(10)

# Nearest Neighbours - Average

In [ ]:
def nb(pc):
    nb_count=3
    nba_list=[5,10,15]
    key_dist={}
    key_idx={}
    #print(len(pc))
    p1i=pc.reset_index().pid
    if len(pc)>np.array(nba_list).max():
        
        dmap=spatial.distance.squareform(spatial.distance.pdist(pc[['xs','ys','zs']].values))
        dmap_sorted=np.sort(dmap, axis=0)
        #dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in nba_list:

            nb_dist=(dmap_sorted[1:(i+1),:]).mean(axis=0)
            #nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nba_d_' + str(i)]=nb_dist
            #key_idx['nb_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        #key_idx.update(key_dist)
    else:
        a = np.empty((len(pc)))
        a[:] = np.nan
        key_dist[('nba_d_' + str(nba_list[0]))]=a
    
    df=pd.DataFrame(key_dist)
    df=pd.concat([p1i, df], axis=1)
    
    return df


In [ ]:
t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nb).reset_index()

In [ ]:
t_grp=t_grp.set_index('pid')

In [ ]:
nba_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)

In [ ]:
nba_grp.head(10)

# Nearest Neighbours -  all colors

def nbc(pc):
    nb_count=3
    key_dist={}
    key_idx={}
    #print(len(pc))
    p1i=pc.reset_index().pid
    if len(pc)>nb_count:
        
        dmap=spatial.distance.squareform(spatial.distance.pdist(pc[['xs','ys','zs']].values))
        dmap_sorted=np.sort(dmap, axis=0)
        dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in range(nb_count):

            nb_dist=(dmap_sorted[i+1,:])
            nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nbc_d_' + str(i)]=nb_dist
            key_idx['nbc_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        key_idx.update(key_dist)
    else:
        a = np.empty((len(pc)))
        a[:] = np.nan
        key_idx['nbc_i_0']=a
        
    
    df=pd.DataFrame(key_idx)
    df=pd.concat([p1i, df], axis=1)
    
    return df


t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nbc).reset_index()

t_grp=t_grp.set_index('pid')

nbc_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)

nbc_grp.sort_index().head()

# Nearest Neighbours -  all colors - Average

def nbca(pc):
    nb_count=3
    nba_list=[5,10,15]
    key_dist={}
    key_idx={}
    #print(len(pc))
    p1i=pc.reset_index().pid
    if len(pc)>np.array(nba_list).max():
        
        dmap=spatial.distance.squareform(spatial.distance.pdist(pc[['xs','ys','zs']].values))
        dmap_sorted=np.sort(dmap, axis=0)
        #dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in nba_list:

            nb_dist=(dmap_sorted[1:(i+1),:]).mean(axis=0)
            #nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nbca_d_' + str(i)]=nb_dist
            #key_idx['nb_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        #key_idx.update(key_dist)
    else:
        a = np.empty((len(pc)))
        a[:] = np.nan
        key_dist[('nbca_d_' + str(nba_list[0]))]=a
    
    df=pd.DataFrame(key_dist)
    df=pd.concat([p1i, df], axis=1)
    
    return df


t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nbca).reset_index()

t_grp=t_grp.set_index('pid')

nbca_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)

nbca_grp.sort_index().head(10)

# Nearest Neighbours -  second color only

p=0
pc1=pc[pc.path==(pd.unique(pc.path)[p])]
#pc1=df[df.c==625]
pc1=pc1[pc1.frame==20]

pd.unique(pc.c)

def nbc2(pc1):
    c1=470
    c2=555

    nb_count=3
    key_dist={}
    key_idx={}


    #print(len(pc))
    if (len(pc1[pc1.c==c1])>nb_count) & (len(pc1[pc1.c==c2])>nb_count):
        pos1=pc1[pc1.c==c1]
        #print(len(pos1))
        p1i=pos1.reset_index().pid
        pos1=pos1[['xs','ys','zs']].values
        pos2=pc1[pc1.c==c2]
        #print(len(pos2))
        p2i=pos2.reset_index().pid
        pos2=pos2[['xs','ys','zs']].values

        dmap=spatial.distance.cdist(pos1, pos2)

        key_dist={}
        key_idx={}
        dmap_sorted=np.sort(dmap.T, axis=0)
        dmap_idx_sorted=np.argsort(dmap.T, axis=0)
        for i in range(nb_count):
            nb_dist=(dmap_sorted[i+1,:])
            nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nbc2_d_' + str(i)]=nb_dist
            key_idx['nbc2_i_' + str(i)]=pd.Series(nb_idx).map(p1i).values.astype('int').tolist()
        key_idx.update(key_dist)
        p1=pd.DataFrame(key_idx)
        p1=pd.concat([p1,p1i], axis=1).set_index('pid')

        key_dist={}
        key_idx={}
        dmap_sorted=np.sort(dmap, axis=0)
        dmap_idx_sorted=np.argsort(dmap, axis=0)
        for i in range(nb_count):
            nb_dist=(dmap_sorted[i+1,:])
            nb_idx=dmap_idx_sorted[i+1,:]
            key_dist['nbc2_d_' + str(i)]=nb_dist
            key_idx['nbc2_i_' + str(i)]=pd.Series(nb_idx).map(p2i).values.astype('int').tolist()
        key_idx.update(key_dist)
        p2=pd.DataFrame(key_idx)
        p2=pd.concat([p2,p2i], axis=1).set_index('pid')
        
        df=pd.concat([p1,p2], axis=0).sort_index()

    else:
        a = np.empty((len(pc1)))
        a[:] = np.nan
        key_idx['nbc2_i_0']=a
        pi=pc1.reset_index().pid
        df=pd.DataFrame(key_idx)
        df=pd.concat([df,pi], axis=1).set_index('pid')
    
    return df


t_grp=pc.set_index('pid').groupby(['path', 'frame']).apply(nbc2)

t_grp=t_grp.reset_index().set_index('pid')

nbc2_grp=t_grp.drop(['path', 'frame'], axis=1)

nbc2_grp.head()

# DBSCAN - is this a measure of object clustering? We have not used this so far but we might find interesting inforamation here especially in the arms. Optimise this code furhter?

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
#from sklearn.datasets.samples_generator import make_blobs
#from sklearn.preprocessing import StandardScaler

def cluster(pc):
    
    
    min_samples=5
    
    eps_list=[5,7.5,10,15,20]
    
    cl_=[]
    
    #X=pos[:,:-1] # 2D
    X=pc[['xs','ys','zs']].values # 3D
    
    for eps in eps_list:
        # Compute DBSCAN
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

        #print('Estimated number of clusters: %d' % n_clusters_)
        
        cl_.append(pd.DataFrame({('cl_idx_' + str(eps) ) : (labels)}))
                     
    

        
    ##############################################################################
    # Plot result

    # Black removed and is used for noise instead.
    
#    unique_labels = set(labels)
#    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
#    for k, col in zip(unique_labels, colors):
#        if k == -1:
#            # Black used for noise.
#            col = 'k'

#        class_member_mask = (labels == k)

#        xy = X[class_member_mask & core_samples_mask]
#       plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
#                 markeredgecolor='k', markersize=8)

#        xy = X[class_member_mask & ~core_samples_mask]
#        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
#                 markeredgecolor='k', markersize=3)

#    plt.title('Estimated number of clusters: %d' % n_clusters_)
#    plt.show()
    cl_.append(pc.reset_index().pid)
    return (pd.concat(cl_, axis=1))

In [ ]:
t_grp=pc.groupby(['path', 'frame']).apply(cluster).reset_index()

In [ ]:
cl_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)


In [ ]:
cl_grp=cl_grp.set_index('pid')

In [ ]:
cl_grp.head(10)

# DBSCAN 2

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
#from sklearn.datasets.samples_generator import make_blobs
#from sklearn.preprocessing import StandardScaler

def cluster_d(pc):
    
    min_samples=5
    
    eps_list=np.arange(3, 31, 2)
    
    cl_=[]
    
    #X=pos[:,:-1] # 2D
    X=pc[['xs','ys','zs']].values # 3D
    
    for eps in eps_list:
        # Compute DBSCAN
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_

        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

        #print('Estimated number of clusters: %d' % n_clusters_)
        
        labels[labels>-1]=eps
        labels[labels==-1]=42
        
               
        cl_.append(pd.DataFrame({('cl_idx_' + str(eps) ) : (labels)}))
    cld_grp=pd.concat(cl_, axis=1)
    cld_grp=pd.DataFrame({('cld' ) : (cld_grp.min(axis=1))})
    
                     
        

        
    ##############################################################################
    # Plot result

    # Black removed and is used for noise instead.
    
#    unique_labels = set(labels)
#    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
#    for k, col in zip(unique_labels, colors):
#        if k == -1:
#            # Black used for noise.
#            col = 'k'

#        class_member_mask = (labels == k)

#        xy = X[class_member_mask & core_samples_mask]
#       plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
#                 markeredgecolor='k', markersize=8)

#        xy = X[class_member_mask & ~core_samples_mask]
#        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=col,
#                 markeredgecolor='k', markersize=3)

#    plt.title('Estimated number of clusters: %d' % n_clusters_)
#    plt.show()
    cld_grp['pid']=(pc.reset_index().pid)
    return (cld_grp)

In [ ]:
t_grp=pc.groupby(['path', 'frame']).apply(cluster_d).reset_index()

In [ ]:
t_grp.tail()

In [ ]:
cld_grp=t_grp.drop(['path', 'frame', 'level_2'], axis=1)


In [ ]:
cld_grp=cld_grp.set_index('pid')

In [ ]:
cld_grp.head(10)

# Stability 2, pc only

In [ ]:
def do_tstab(tgrp):
    ocp_=[]
    first=True
    #print(len(tgrp))
    for i, grp in tgrp.groupby(['frame']):
        pos=grp[['xs','ys','zs']].values
        if first:
            first=False
        else:
            dmap=spatial.distance.cdist(t1, pos)
            dmap_sorted=np.sort(dmap, axis=1)
            data=dmap_sorted[:,0]

            ocp_.append(pd.DataFrame({'stab' : data}))#, 'pid': grp.pid}))

        t1=pos
    data=np.zeros(len(pos))
    data[:]=np.nan
    ocp_.append(pd.DataFrame({'stab' : data}))#, 'pid':grp.pid}))

    ocp=pd.concat(ocp_, axis=0).reset_index()
    ocp['pid']=tgrp.reset_index().pid
    
    return ocp

In [ ]:
t_grp=pc.groupby(['path']).apply(do_tstab)

In [ ]:
stab_grp=t_grp.reset_index()[['stab', 'pid']]

In [ ]:
stab_grp=stab_grp.set_index('pid').sort_index()

In [ ]:
stab_grp.head(10)

# Point depth

def depth(pos):
    
    fill_dist=15
    
    pos_max=pos[['xs', 'ys', 'zf']].max()

    zsize=int(pos_max['zf']*1.5)
    xsize=int(pos_max['xs']*1.1)
    ysize=int(pos_max['ys']*1.1)

    pcc=np.zeros((xsize+2,ysize+2,zsize+2))
    pc_pos=pos[['xs', 'ys', 'zf']].values

    #zfloor=int(np.percentile(pc_pos[:,2],2))
    zfloor=1
    pc_pos[pc_pos<0]=0
    pc_pos=pc_pos.astype('int').T
    
    pc_pos=pc_pos.tolist()
      
    pcc[pc_pos]=1
    pcc[:,:,:zfloor]=1

    pcd=ndimage.morphology.distance_transform_edt(pcc==0)

    #plt.figure(figsize=(10,3))
    #plt.imshow((pcd[300,:,:]).T, cmap='jet', vmin=0, vmax=20)
    #plt.colorbar()

    pca=pcd<fill_dist

    pca=ndimage.binary_fill_holes(pca)

    #plt.figure(figsize=(10,3))
    #plt.imshow((pca[300,:,:]).T, cmap='jet', vmin=0, vmax=2)
    #plt.colorbar()

    pcad=ndimage.morphology.distance_transform_edt(pca)

    #plt.figure(figsize=(10,3))
    #plt.imshow((pcad[300,:,:]).T, cmap='jet', vmin=0, vmax=40)
    #plt.colorbar()

    pdepth=pcad[pc_pos]-fill_dist
           
    depth_grp=pd.DataFrame({'depth' : (pdepth)})
    
    depth_grp['pid']=pos.reset_index().pid
    
    return depth_grp


def depth(pos_grp):
    
    fill_dist=15
    
    pos_max=pos_grp[['xs', 'ys', 'zf']].max()

    zsize=int(pos_max['zf']*1.5)
    xsize=int(pos_max['xs']*1.1)
    ysize=int(pos_max['ys']*1.1)

    depth_grp_=[]
    
    for i, pos in pos_grp.groupby('frame'):
    
        pcc=np.zeros((xsize+2,ysize+2,zsize+2))
        
        pc_pos=pos[['xs', 'ys', 'zf']].values
        
        zfloor=1
        pc_pos[pc_pos<0]=0
        pc_pos=pc_pos.astype('int').T

        pc_pos=pc_pos.tolist()

        pcc[pc_pos]=1
        pcc[:,:,:zfloor]=1

        pcd=ndimage.morphology.distance_transform_edt(pcc==0)
        pca=pcd<fill_dist

        pca=ndimage.binary_fill_holes(pca)
        pcad=ndimage.morphology.distance_transform_edt(pca)
        pdepth=pcad[pc_pos]-fill_dist

        depth_grp=pd.DataFrame({'depth' : (pdepth)})

        depth_grp['pid']=pos.reset_index().pid
        
        depth_grp_.append(depth_grp)
    
    depth_grp_=pd.concat(depth_grp_, axis=0)
    return depth_grp_


t_grp=pc.groupby(['path']).apply(depth).reset_index()   

depth_grp=t_grp[['pid', 'depth']]

depth_grp=depth_grp.set_index('pid').sort_index()

depth_grp.tail(10)

# Export

In [ ]:
# Pia - change base name to current folder. 
# Pia Make sure there is no subfolder called mods if the folder was copied from other experiment??? We are making a mods subfolder
base_name = '200823_Test'

In [ ]:
folder='./mods/'
#/tmp/year
os.mkdir(folder)
base_name=folder + base_name 

In [ ]:
# Saving modified point-cloud data as pickle file: file name pc_mod
#pc=pc.set_index('pid')
pc.to_pickle(base_name + '_pc_mod')
print(len(pc))
pc.head()

In [ ]:
# Saving stabiliy data as pickle file:  file name _stab
stab_grp.to_pickle(base_name + '_stab')
print(len(stab_grp))
stab_grp.head()

In [ ]:
cl_grp.sort_index().to_pickle(base_name + '_cl')
print(len(cl_grp))
cl_grp.sort_index().tail()

In [ ]:
cld_grp.sort_index().to_pickle(base_name + '_cld')
print(len(cld_grp))
cld_grp.sort_index().tail()

In [ ]:
# Saving tracks as pickle file: file name_tracks
#tracks_grp=tracks_grp.set_index('pid')
tracks_grp.to_pickle(base_name + '_tracks')
print(len(tracks_grp))
tracks_grp.head()

depth_grp.to_pickle(base_name + '_depth')
print(len(depth_grp))
depth_grp.head()

## Neighbours

In [ ]:
# Saving nearest neighbours data as pickle: file name_nb
nb_grp.to_pickle(base_name + '_nb')
print(len(nb_grp))
nb_grp.tail()

nbc_grp.to_pickle(base_name + '_nbc')
print(len(nbc_grp))
nbc_grp.tail()

In [ ]:
# Saving nearest neighbours average data as pickle: file name_nba
nba_grp.to_pickle(base_name + '_nba')
print(len(nba_grp))
nba_grp.tail()

nbca_grp.to_pickle(base_name + '_nbca')
print(len(nbca_grp))
nbca_grp.tail()

nbc2_grp.to_pickle(base_name + '_nbc2')
print(len(nbc2_grp))
nbc2_grp.tail()

# Addon tracking - contraction component (förstör pc)

In [ ]:
#pc=pc.reset_index()
pc.head()

In [ ]:
pc1=pd.concat([pc[['path', 'xs', 'ys', 'zf', 'pid','frame']].set_index('pid'), tracks_grp], axis=1).sort_index()

In [ ]:
def contract(t2):
    #xmed, ymed =(t2[t2.frame<50]['xs'].median(), t2[t2.frame<50]['ys'].median())
    #xmed, ymed =(t2['xs'].median(), t2['ys'].median())

    #t2['xm']=t2['xs']-xmed
    #t2['ym']=t2['ys']-ymed

    #t2['cont']=((-t2['xm'])*t2['dvx'] + (-t2['ym'])*t2['dvy'] + (-t2['zf'])*t2['dvz'] )/((t2['xm'])**2 + (t2['ym'])**2 + (t2['zf'])**2)**0.5
    t2['cont']=((-t2['xs'])*t2['dvx'] + (-t2['ys'])*t2['dvy'] + (-t2['zf'])*t2['dvz'] )/((t2['xs'])**2 + (t2['ys'])**2 + (t2['zf'])**2)**0.5
    t2['cont_p']=t2.cont/t2.dv
    
    
    
    return pd.DataFrame({'cont' : (t2['cont']), 'cont_p' : (t2['cont_p']), 'pid':t2['pid']})

In [ ]:
cont_grp=pc1.reset_index().groupby(['path']).apply(contract)
cont_grp=cont_grp.set_index('pid').sort_index()

In [ ]:
cont_grp.to_pickle(base_name + '_cont')
print(len(cont_grp))
cont_grp.head()

# Nytt paket med lite nya numeriska variabler som beskriver tracking och totala rorelser 

## These functions give us the extra readouts incluidng tracknr, nrtracks, cont_tot, disp_tot, dvz_tot

In [ ]:
#Skapar dataframe med grundpaket plus tracking plus kontraktions
pc1=pd.concat([pc[['path', 'xs', 'ys', 'zf', 'pid','frame']].set_index('pid'), tracks_grp,cont_grp], axis=1).sort_index()

In [ ]:
pc1.head()

In [ ]:
#Satter index pa path och particle for att kunna konkaternera dataframe senare
pc1=pc1.reset_index().set_index(['path','particle'])

#Grupperar data pa path och partikel for att kunna analysera olika partiklar for sig
grouped=pc1.groupby(['path','particle'])
#Raknar antalet observationer for varje unik partikel
counted=grouped.count()
#Variabeln nrtracks kollar hur manga tracks en viss partikel ar trackad
pc1['nrtracks']=counted.frame

#Adderar de olika variablerna och beraknar summan av alla observationer
summed=grouped.sum()
#cont_tot summerar den totala kontraktionen for en partikel
pc1['cont_tot']=summed.cont
# displ_tot summerar den totala forflyttningen for en partikel
pc1['displ_tot']=abs(summed.dvx)+abs(summed.dvy)
# dvz_tot summerar den totala forflyttningen i z-led for en partikel
pc1['dvz_tot']=summed.dvz

In [ ]:
#Satter om index till pid for att ha ett unikt index for varje observation
pc1=pc1.reset_index().set_index('pid')
# Tracknr raknar vilken trackning i ordningen en viss observation ar
tracknr=pc1.groupby(['path','particle'])['frame'].rank()
pc1['tracknr']=tracknr

#Kontrollerar att de nya variablerna fungerar som de ska
pc1.loc[pc1.nrtracks>5,['path','particle','frame','tracknr','nrtracks','cont_tot','displ_tot','dvz_tot']].set_index(['path','particle','frame']).sort_index().head(20)

In [ ]:
# Saving these readouts as pickle file: file name_xtratracking
xtra_tracking=pc1.loc[:,['tracknr','nrtracks','cont_tot','displ_tot','dvz_tot']]
xtra_tracking.to_pickle(base_name + '_xtratracking')
print(len(xtra_tracking))
xtra_tracking.head()

In [ ]:
os.getcwd()